In [7]:
import yaml
import torch
import argparse
import numpy as np
import open3d as o3d
import torchvision.transforms as T
from copy import deepcopy
import cv2
from easydict import EasyDict as edict

import plotly.graph_objects as go

# o3d.visualization.webrtc_server.enable_webrtc()
# 在notebook中使用open3d可视化需要启用webrtc_server
# 貌似这个会自动设置，不能手动二次指定

# test_color = "/home/haoxiang/RISE-2/saved_test_data/color_0000.png"
# test_depth = "/home/haoxiang/RISE-2/saved_test_data/depth_0000.png"
test_color = "/data/haoxiang/data/FLIPPING_v3/train/scene_0001/cam_104122060902/color/1767593840262.png"
test_depth = "/data/haoxiang/data/FLIPPING_v3/train/scene_0001/cam_104122060902/depth/1767593840262.png"

config_path = "test.yaml"

fake_intrinsics = np.array([
    [915.384521484375, 0.0, 633.3715209960938],
    [0.0, 914.9421997070312, 354.1505432128906],
    [0.0, 0.0, 1.0]
])


fake_depth_scale = 1000.0

def load_test_obs(color_path, depth_path):
    # 1. 加载彩色图并转为 RGB (OpenCV 默认读入是 BGR)
    color_image = cv2.imread(color_path)
    if color_image is None:
        raise ValueError(f"无法加载图片: {color_path}")
    color_image = cv2.cvtColor(color_image, cv2.COLOR_BGR2RGB).astype(np.uint8)

    # 2. 加载深度图
    # 注意：必须使用 cv2.IMREAD_UNCHANGED 才能保留 16bit 深度信息
    depth_image = cv2.imread(depth_path, cv2.IMREAD_UNCHANGED)
    if depth_image is None:
        raise ValueError(f"无法加载深度图: {depth_path}")
    
    # 确保是 uint16。如果你的 PNG 是 8bit 的，需要根据量化比例转回 uint16 (通常单位是毫米)
    depth_image = depth_image.astype(np.uint16)

    return color_image, depth_image

def resize_image(image_list, image_size, interpolation = T.InterpolationMode.BILINEAR):
    resize = T.Resize(image_size, interpolation)
    image_list_resized = resize(image_list)
    return image_list_resized

def create_point_cloud(colors, depths, intrinsics, config, depth_scale = 1000.0, rescale_factor = 1):
    """
    color, depth => point cloud
    """
    if rescale_factor != 1:
        H, W = depths.shape
        h, w = int(H * rescale_factor), int(W * rescale_factor)
        colors = colors.transpose([2, 0, 1]).astype(np.float32)
        colors = torch.from_numpy(colors)
        colors = np.ascontiguousarray(resize_image(colors, [h, w]).numpy().transpose([1, 2, 0]))
        depths = depths.astype(np.float32)
        depths = torch.from_numpy(depths[np.newaxis])
        depths = resize_image(depths, [h,w], interpolation = T.InterpolationMode.NEAREST)[0]
        depths = depths.numpy()

    # generate point cloud
    h, w = depths.shape
    fx, fy = intrinsics[0, 0] * rescale_factor, intrinsics[1, 1] * rescale_factor
    cx, cy = intrinsics[0, 2] * rescale_factor, intrinsics[1, 2] * rescale_factor
    colors = o3d.geometry.Image(colors.astype(np.uint8))
    depths = o3d.geometry.Image(depths.astype(np.float32))
    camera_intrinsics = o3d.camera.PinholeCameraIntrinsic(
        width = w, height = h, fx = fx, fy = fy, cx = cx, cy = cy
    )
    rgbd = o3d.geometry.RGBDImage.create_from_color_and_depth(
        colors, depths, depth_scale, convert_rgb_to_intensity = False
    )
    cloud = o3d.geometry.PointCloud.create_from_rgbd_image(rgbd, camera_intrinsics)
    # crop point cloud
    bbox3d = o3d.geometry.AxisAlignedBoundingBox(config.deploy.workspace.min, config.deploy.workspace.max)
    cloud = cloud.crop(bbox3d)
    # downsample
    cloud = cloud.voxel_down_sample(config.data.voxel_size)
    return cloud

def create_input(colors, depths, cam_intrinsics, config, depth_scale = 1000.0, rescale_factor = 1):
    """
    colors, depths => coords, points
    """
    # create point cloud
    cloud = create_point_cloud(
        colors, 
        depths, 
        cam_intrinsics, 
        config,
        depth_scale = depth_scale,
        rescale_factor = rescale_factor,
    )

    # convert to sparse tensor
    points = np.asarray(cloud.points)
    coords = np.ascontiguousarray(points / config.data.voxel_size, dtype = np.int32)

    return coords, points, cloud

def visualize_in_jupyter(cloud):
    points = np.asarray(cloud.points)
    colors = np.asarray(cloud.colors)
    
    if len(points) > 100000: # 采样点数可以稍微设高一点，Plotly 在 10w 点左右还算流畅
        idx = np.random.choice(len(points), 100000, replace=False)
        points = points[idx]
        colors = colors[idx]

    fig = go.Figure(data=[go.Scatter3d(
        x=points[:, 0], y=points[:, 1], z=points[:, 2],
        mode='markers',
        marker=dict(
            size=1.5,      # 增大点的大小，看起来更实一点
            color=colors, 
            opacity=1.0
        )
    )])
    
    # --- 修改这里来增大窗口 ---
    fig.update_layout(
        # width=1000,        # 设置宽度（像素）
        height=800,        # 设置高度（像素）
        margin=dict(l=0, r=0, b=0, t=0), # 去掉四周留白，让点云撑满窗口
        scene=dict(
            aspectmode='data', # 保持物理坐标比例
            xaxis=dict(visible=True), # 如果不想看轴，可以设为 False
            yaxis=dict(visible=True),
            zaxis=dict(visible=True),
        )
    )
    
    fig.show()

# load config
with open(config_path, "r") as f:
    config = edict(yaml.load(f, Loader = yaml.FullLoader))
config.data.normalization.trans_min = np.asarray(config.data.normalization.trans_min)
config.data.normalization.trans_max = np.asarray(config.data.normalization.trans_max)

colors, depths = load_test_obs(test_color, test_depth)

coords, points, cloud = create_input(
    colors,
    depths,
    # cam_intrinsics = agent.intrinsics,
    cam_intrinsics = fake_intrinsics,
    config = config,
    # depth_scale = agent.camera.depth_scale,
    depth_scale = fake_depth_scale,
    rescale_factor = 1.0
)

# o3d.visualization.draw([cloud])
visualize_in_jupyter(cloud)